In [ ]:
# If run experiment on Google Colab, upload all code to Google Drive, and run this chunk to set the working directory to where the code is located at.
# Otherwise, just set the working directory to your local directory where you store the code. 

# Set the working directory to a folder in my Google Drive. 
import os 
# the base Google Drive directory
root_dir = "/content/drive/My Drive/"

# set it to where your project files are saved
project_folder = "mirror_experiment"

def create_and_set_working_directory(project_folder):
  # check if the project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use the project folder as the working directory
  os.chdir(root_dir + project_folder)

  print('\nYour working directory was changed to ' + root_dir + project_folder )

create_and_set_working_directory(project_folder)

In [ ]:
#### import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import dist_img_m, synthetic_img_input
from algos import PDASMD_w_rounding, APDAGD_w_rounding, Sinkhorn_w_rounding, AAM_w_rounding
from algos import Stochastic_sinkhorn_w_rounding, APDRCD_w_rounding
from scipy.stats import linregress
import pickle
from PIL import Image


#%% import mnist images
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#1. Compare PDASMD with other algorithms on synthetic data


In [ ]:
#%% generating the images
alpha_list = []
beta_list = []
C_list = []
m_list = [3,5,7,9,12]
size_list = list(map(lambda x: x**2,m_list))
reps_each_m = 5
num_m = len(m_list)
np.random.seed(1010)
seed_list_1 = np.random.choice(range(reps_each_m*100),reps_each_m)
seed_list_2 = np.random.choice(range(reps_each_m*100,reps_each_m*200),reps_each_m)

for m in m_list:
    for seed1,seed2 in zip(seed_list_1,seed_list_2):
        alpha_list.append( synthetic_img_input(m, seed = seed1)  ) 
        beta_list.append( synthetic_img_input(m, seed = seed2)  ) 
    C_list.append(dist_img_m(m))

#%%% parameter list
epsilons = [.05] * reps_each_m * num_m
sizes = np.repeat(size_list, reps_each_m)
idxes_c = np.repeat(range(num_m), reps_each_m)
idxes_dist = range(num_m * reps_each_m)
settings = list(zip(epsilons,sizes,idxes_c,idxes_dist))

#%% example images to take a look
m_list = [7,9,11,12]

fig,ax = plt.subplots(2,2,figsize = (4,4))
#plt.tight_layout()
ax = ax.ravel()
for i,m in enumerate(m_list):
    temp = synthetic_img_input(m,seed = 5735)
    image = Image.fromarray(np.array(temp.reshape(m,m)*255/max(temp),np.uint8))
    ax[i].imshow(image,cmap = 'Blues')
    ax[i].set_title(r"{} * {}".format(m,m))
    ax[i].axis('off')
  
plt.savefig("results/sample_image.pdf", format="pdf", bbox_inches = 'tight')


In [ ]:
#%% calculate epsilon solution, the goal is to compare the total computation
PDASMD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, 
        inner_iters = size, MD = True)
    PDASMD_result.append([solution, computations, time])

#%% 
PDASGD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, 
        inner_iters = size, MD = False)
    PDASGD_result.append([solution, computations, time])
#%%
APDAGD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = APDAGD_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    APDAGD_result.append([solution, computations, time])
#%%
AAM_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = AAM_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    AAM_result.append([solution, computations, time])
#%%
Sinkhorn_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = Sinkhorn_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    Sinkhorn_result.append([solution, computations, time])

#%% Uncomment to run APDRCD, it is pretty slow.
#APDRCD_result = []
#i = 1
#for epsilon, size, idx_c, idx_dist in settings:
#    print("running {}/{} experiment".format(i,len(settings)))
#    i += 1
#    solution, computations, time = APDRCD_w_rounding(
#        epsilon = epsilon, Size = size, C = C_list[idx_c],
#        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
#    APDRCD_result.append([solution, computations, time])
    
#%%
Stoc_Sinkhorn_result = []

i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = Stochastic_sinkhorn_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    Stoc_Sinkhorn_result.append([solution, computations, time])


In [ ]:
#%%plot the results
PDASMD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(PDASMD_result,np.repeat(size_list, reps_each_m))])
PDASMD_result_cleaned.columns = ['computations','time','n']
#%%
APDAGD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(APDAGD_result,np.repeat(size_list, reps_each_m))])
APDAGD_result_cleaned.columns = ['computations','time','n']

AAM_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(AAM_result,np.repeat(size_list, reps_each_m))])
AAM_result_cleaned.columns = ['computations','time','n']

Sinkhorn_result_cleaned = pd.DataFrame([[i[0][1], i[0][2],i[1]] for i in zip(Sinkhorn_result,np.repeat(size_list, reps_each_m))])
Sinkhorn_result_cleaned.columns = ['computations','time','n']

#%% plot the comparison with deterministic algorithms

plt.figure(dpi=1000)


lr_smd  = linregress(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASMD, slope = {:.2f}".format(lr_smd[0]),color='darkgreen', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASMD_result_cleaned.groupby('n').max()['computations']),
         color='darkgreen', alpha = 0.2)

lr_gd  = linregress(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').mean()['computations']),label="APDAGD, slope = {:.2f}".format(lr_gd[0]),color='pink', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').min()['computations']),
         np.log(APDAGD_result_cleaned.groupby('n').max()['computations']),
         color='pink', alpha = 0.2)

lr_aam  = linregress(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').mean()['computations']),label="AAM, slope = {:.2f}".format(lr_aam[0]),color='blue', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').min()['computations']),
         np.log(AAM_result_cleaned.groupby('n').max()['computations']),
         color='blue', alpha = 0.2)

lr_sin  = linregress(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').mean()['computations']),label="Sinkhorn, slope = {:.2f}".format(lr_sin[0]),color='violet', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').min()['computations']),
         np.log(Sinkhorn_result_cleaned.groupby('n').max()['computations']),
         color='violet', alpha = 0.2)
#lr_gd  = linregress(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']))
#plt.plot(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']),label="APDAGD, slope = {:.2f}".format(lr_gd[0]),color='crimson', marker ='*',markersize = 3,lw=1)
#plt.fill_between(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').min()['computations']),
#         np.log(PDASGD_result_cleaned.groupby('n').max()['computations']),
#         color='crimson', alpha = 0.2)

plt.xlabel(r'$ln(n)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=13)
plt.title("ln(number of operations) versus ln(n)",fontsize=20)

#plt.show()

plt.savefig("results/sys_compare_deter.pdf", format="pdf",bbox_inches = 'tight')



#%% plot the comparison with stochastic algorithms

PDASGD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(PDASGD_result,np.repeat(size_list, reps_each_m))])
PDASGD_result_cleaned.columns = ['computations','time','n']

#APDRCD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(APDRCD_result,np.repeat(size_list, reps_each_m))])
#APDRCD_result_cleaned.columns = ['computations','time','n']

Sto_Sink_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(Stoc_Sinkhorn_result,np.repeat(size_list, reps_each_m))])
Sto_Sink_result_cleaned.columns = ['computations','time','n']
#%%

plt.figure(dpi=1000)


lr_smd  = linregress(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASMD, slope = {:.2f}".format(lr_smd[0]),color='darkgreen', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASMD_result_cleaned.groupby('n').max()['computations']),
         color='darkgreen', alpha = 0.2)

lr_sgd  = linregress(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASGD, slope = {:.2f}".format(lr_sgd[0]),color='pink', marker ='*',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASGD_result_cleaned.groupby('n').max()['computations']),
         color='pink', alpha = 0.2)

#lr_rcd  = linregress(np.log(size_list), np.log(APDRCD_result_cleaned.groupby('n').mean()['computations']))
#plt.plot(np.log(size_list), np.log(APDRCD_result_cleaned.groupby('n').mean()['computations']), label="APDRCD, slope = {:.2f}".format(lr_rcd[0]),color='blue', marker ='',markersize = 6,lw=1)
#plt.fill_between(np.log(size_list), np.log(APDRCD_result_cleaned.groupby('n').min()['computations']),
#         np.log(APDRCD_result_cleaned.groupby('n').max()['computations']),
#         color='blue', alpha = 0.2)

lr_ss  = linregress(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').mean()['computations']),
         label="Stochastic Sinkhorn, slope = {:.2f}".format(lr_ss[0]),color='violet', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').min()['computations']),
         np.log(Sto_Sink_result_cleaned.groupby('n').max()['computations']),
         color='violet', alpha = 0.2)


plt.xlabel(r'$ln(n)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=13)
plt.title("ln(number of operations) versus ln(n)",fontsize=20)

#plt.show()

plt.savefig("results/sys_compare_stoc.pdf", format="pdf",bbox_inches = 'tight')

#%% save the data and results

results = {'alpha_list': alpha_list, 'beta_list': beta_list, 'C_list': C_list, 
    'PDASMD_result': PDASMD_result, 'PDASGD_result': PDASGD_result, 
           'APDAGD_result': APDAGD_result, 'AAM_result': AAM_result, 
           'Sinkhorn_result': Sinkhorn_result, 'Stoc_Sinkhorn_result': Stoc_Sinkhorn_result}
  
try:
    file = open('results/comparison_syn', 'wb')
    pickle.dump(results,file)
    file.close()
  
except:
    print("Something went wrong")


#2. Evaluate the performance of PDASMD-B on synthetic data


In [ ]:
#%% generating the images
alpha_list = []
beta_list = []
C_list = []
m_list = [8,10,12]
size_list = list(map(lambda x: x**2,m_list))
reps_each_m = 5
num_m = len(m_list)
np.random.seed(1010)
seed_list_1 = np.random.choice(range(reps_each_m*100),reps_each_m)
seed_list_2 = np.random.choice(range(reps_each_m*100,reps_each_m*200),reps_each_m)

for m in m_list:
    for seed1,seed2 in zip(seed_list_1,seed_list_2):
        alpha_list.append( synthetic_img_input(m, seed = seed1)  ) 
        beta_list.append( synthetic_img_input(m, seed = seed2)  ) 
    C_list.append(dist_img_m(m))

#%%% parameter list
epsilons = [.05] * reps_each_m * num_m
sizes = np.repeat(size_list, reps_each_m)
idxes_c = np.repeat(range(num_m), reps_each_m)
idxes_dist = range(num_m * reps_each_m)

batch_sizes = [i for i in range(2,49,8)]

settings = []
for epsilon, size, idx_c, idx_dist in list(zip(epsilons,sizes,idxes_c,idxes_dist)):
    for batch_size in batch_sizes:
        settings.append([epsilon, size, idx_c, idx_dist, batch_size])

In [ ]:
#%% calculate epsilon solution, the goal is to compare the total computation
PDASMD_result_batch = []
i = 1
for epsilon, size, idx_c, idx_dist, batch_size in settings:
    print("Running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, 
        inner_iters = size//batch_size, batchsize = batch_size, MD = True)
    PDASMD_result_batch.append([solution, computations, time])

In [ ]:
#%%plot the results
PDASMD_result_batch_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1][1],i[1][4]] for i in zip(PDASMD_result_batch,settings)])
PDASMD_result_batch_cleaned.columns = ['computations','time','n','batchsize']

#%%
plt.figure(dpi=1000)

PDASMD_result_batch_mean = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).mean()['computations'])
PDASMD_result_batch_min = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).min()['computations'])
PDASMD_result_batch_max = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).max()['computations'])
#%%
for n in size_list:
    lr  = linregress(np.log(batch_sizes),PDASMD_result_batch_mean.loc[n] )
    
    plt.plot(np.log(batch_sizes), PDASMD_result_batch_mean.loc[n],label="n = {}, slope = {:.2f}".format(n,lr[0]), marker ='o',markersize = 6,lw=1)
    plt.fill_between(np.log(batch_sizes),PDASMD_result_batch_min.loc[n] ,
             PDASMD_result_batch_max.loc[n], alpha = 0.2)

plt.xlabel(r'$ln(batchsize)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=14)
plt.title("ln(number of operations) versus ln(batchsize)",fontsize=20)

plt.savefig("results/sys_batch_computation.pdf", format="pdf", bbox_inches = 'tight')

#%%

plt.figure(dpi=1000)


PDASMD_result_batch_mean = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).mean()['time'])
PDASMD_result_batch_min = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).min()['time'])
PDASMD_result_batch_max = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).max()['time'])

for n in size_list:
    #lr  = linregress(np.log(batch_sizes),PDASMD_result_batch_mean.loc[n] )
    
    plt.plot(np.log(batch_sizes), PDASMD_result_batch_mean.loc[n],label="n = {}".format(n), marker ='o',markersize = 6,lw=1)
    plt.fill_between(np.log(batch_sizes),PDASMD_result_batch_min.loc[n] ,
             PDASMD_result_batch_max.loc[n], alpha = 0.2)

plt.xlabel(r'$ln(batchsize)$',fontsize=20)
plt.ylabel(r'$ln(running time)$',fontsize=20) 
plt.legend(fontsize=20)
plt.title("ln(running time) versus ln(batchsize)",fontsize=20)


plt.savefig("results/sys_batch_time.pdf", format="pdf", bbox_inches = 'tight')

#%% save the data and results

results = {'alpha_list': alpha_list, 'beta_list': beta_list, 'C_list': C_list, 
    'PDASMD_result_batch': PDASMD_result_batch}
  
try:
    file = open('results/comparison_syn_batch', 'wb')
    pickle.dump(results,file)
    file.close()
  
except:
    print("Something went wrong")
    
#%% Retrive the data
results_file = open('results/comparison_syn_batch', 'rb')     
results = pickle.load(results_file)
PDASMD_result_batch = results['PDASMD_result_batch']

#3. Compare PDASMD with other algorithms on MNIST data


In [ ]:
### experiment setup
alpha_list = []
beta_list = []
C_list = []
m_list = [2,3,4,5,6,7,8,9,10]
size_list = list(map(lambda x: x**2,m_list))
reps_each_m = 5
num_m = len(m_list)
#### randomly sample from training images as the marginals
#np.random.seed(3894)
np.random.seed(1001)
idx_list_1 = np.random.choice(range(30000),reps_each_m)
idx_list_2 = np.random.choice(range(30000,60000),reps_each_m)

#### marginal distribution list, where we reshape the mnist image to get different size marginals
for m in m_list:
    for idx1,idx2 in zip(idx_list_1,idx_list_2):
      fig1 = Image.fromarray(x_train[idx1]).resize((m,m)) ### down scale the images
      fig2 = Image.fromarray(x_train[idx2]).resize((m,m)) 
      fig1 = np.array(fig1, np.float64)
      fig2 = np.array(fig2, np.float64) 
      idx = np.where(fig1 <= .15* np.max(fig1)) 
      fig1[idx] = .15* np.max(fig1) ### add background intensity
      idx = np.where(fig2 <= .15* np.max(fig2)) 
      fig2[idx] = .15* np.max(fig2) ### add background intensity
      fig1 /= fig1.sum()
      fig2 /= fig2.sum()

      alpha_list.append(fig1.reshape(-1,1)) 
      beta_list.append(fig2.reshape(-1,1)) 
    
    C_list.append(dist_img_m(m))

#%%% parameter list
epsilons = [.1] * reps_each_m * num_m
sizes = np.repeat(size_list, reps_each_m)
idxes_c = np.repeat(range(num_m), reps_each_m)
idxes_dist = range(num_m * reps_each_m)
settings = list(zip(epsilons,sizes,idxes_c,idxes_dist))


In [ ]:
#### an example of down scaled image with added background intensity
fig1 = Image.fromarray(x_train[idx1]).resize((m,m))
idx = np.where(fig1 <= .15* np.max(fig1)) 
fig1 = np.array(fig1, np.float64)
fig1[idx] = (.15* np.max(fig1))
Image.fromarray(np.array(fig1,np.uint8)).resize((50,50))

In [ ]:
#%% calculate epsilon solution, the goal is to compare the total computation
PDASMD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("Running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, inner_iters = size, MD = True)
    PDASMD_result.append([solution, computations, time])

In [ ]:
PDASGD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, 
        inner_iters = size, MD = False)
    PDASGD_result.append([solution, computations, time])

In [ ]:
APDAGD_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = APDAGD_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    APDAGD_result.append([solution, computations, time])
#%%

In [ ]:

AAM_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = AAM_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    AAM_result.append([solution, computations, time])
#%%

In [ ]:
Sinkhorn_result = []
i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = Sinkhorn_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    Sinkhorn_result.append([solution, computations, time])


In [ ]:

#%% skip APDRCD since it is too slow
#APDRCD_result = []
#i = 1
#for epsilon, size, idx_c, idx_dist in settings:
#    print("running {}/{} experiment".format(i,len(settings)))
#    i += 1
#    solution, computations, time = APDRCD_w_rounding(
#        epsilon = epsilon, Size = size, C = C_list[idx_c],
#        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
#    APDRCD_result.append([solution, computations, time])
    
#%%
Stoc_Sinkhorn_result = []

i = 1
for epsilon, size, idx_c, idx_dist in settings:
    print("running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = Stochastic_sinkhorn_w_rounding(
        epsilon = epsilon, Size = size, C = C_list[idx_c],
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist])
    Stoc_Sinkhorn_result.append([solution, computations, time])

In [ ]:
#%% save the data and results

results = {'alpha_list': alpha_list, 'beta_list': beta_list, 'C_list': C_list, 
    'PDASMD_result': PDASMD_result, 'PDASGD_result': PDASGD_result, 
           'APDAGD_result': APDAGD_result, 'AAM_result': AAM_result, 
           'Sinkhorn_result': Sinkhorn_result, 'Stoc_Sinkhorn_result': Stoc_Sinkhorn_result}
  
try:
    file = open('results/comparison_mnist', 'wb')
    pickle.dump(results,file)
    file.close()
  
except:
    print("Something went wrong")

In [ ]:
#%%plot the results
PDASMD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(PDASMD_result,np.repeat(size_list, reps_each_m))])
PDASMD_result_cleaned.columns = ['computations','time','n']
#%%
APDAGD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(APDAGD_result,np.repeat(size_list, reps_each_m))])
APDAGD_result_cleaned.columns = ['computations','time','n']

AAM_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(AAM_result,np.repeat(size_list, reps_each_m))])
AAM_result_cleaned.columns = ['computations','time','n']

Sinkhorn_result_cleaned = pd.DataFrame([[i[0][1], i[0][2],i[1]] for i in zip(Sinkhorn_result,np.repeat(size_list, reps_each_m))])
Sinkhorn_result_cleaned.columns = ['computations','time','n']

#%% plot the comparison with deterministic algorithms

plt.figure(dpi=1000)


lr_smd  = linregress(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASMD, slope = {:.2f}".format(lr_smd[0]),color='darkgreen', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASMD_result_cleaned.groupby('n').max()['computations']),
         color='darkgreen', alpha = 0.2)

lr_gd  = linregress(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').mean()['computations']),label="APDAGD, slope = {:.2f}".format(lr_gd[0]),color='pink', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(APDAGD_result_cleaned.groupby('n').min()['computations']),
         np.log(APDAGD_result_cleaned.groupby('n').max()['computations']),
         color='pink', alpha = 0.2)

lr_aam  = linregress(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').mean()['computations']),label="AAM, slope = {:.2f}".format(lr_aam[0]),color='blue', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(AAM_result_cleaned.groupby('n').min()['computations']),
         np.log(AAM_result_cleaned.groupby('n').max()['computations']),
         color='blue', alpha = 0.2)

lr_sin  = linregress(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').mean()['computations']),label="Sinkhorn, slope = {:.2f}".format(lr_sin[0]),color='violet', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(Sinkhorn_result_cleaned.groupby('n').min()['computations']),
         np.log(Sinkhorn_result_cleaned.groupby('n').max()['computations']),
         color='violet', alpha = 0.2)
#lr_gd  = linregress(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']))
#plt.plot(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']),label="APDAGD, slope = {:.2f}".format(lr_gd[0]),color='crimson', marker ='*',markersize = 3,lw=1)
#plt.fill_between(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').min()['computations']),
#         np.log(PDASGD_result_cleaned.groupby('n').max()['computations']),
#         color='crimson', alpha = 0.2)

plt.xlabel(r'$ln(n)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=13)
plt.title("ln(number of operations) versus ln(n)",fontsize=20)

#plt.show()

plt.savefig("results/mnist_compare_deter.pdf", format="pdf",bbox_inches = 'tight')


In [ ]:
#%% plot the comparison with stochastic algorithms

PDASGD_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(PDASGD_result,np.repeat(size_list, reps_each_m))])
PDASGD_result_cleaned.columns = ['computations','time','n']


Sto_Sink_result_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1]] for i in zip(Stoc_Sinkhorn_result,np.repeat(size_list, reps_each_m))])
Sto_Sink_result_cleaned.columns = ['computations','time','n']
#%%

plt.figure(dpi=1000)


lr_smd  = linregress(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASMD, slope = {:.2f}".format(lr_smd[0]),color='darkgreen', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASMD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASMD_result_cleaned.groupby('n').max()['computations']),
         color='darkgreen', alpha = 0.2)

lr_sgd  = linregress(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').mean()['computations']),
         label="PDASGD, slope = {:.2f}".format(lr_sgd[0]),color='pink', marker ='*',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(PDASGD_result_cleaned.groupby('n').min()['computations']),
         np.log(PDASGD_result_cleaned.groupby('n').max()['computations']),
         color='pink', alpha = 0.2)


lr_ss  = linregress(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').mean()['computations']))
plt.plot(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').mean()['computations']),
         label="Stochastic Sinkhorn, slope = {:.2f}".format(lr_ss[0]),color='violet', marker ='o',markersize = 6,lw=1)
plt.fill_between(np.log(size_list), np.log(Sto_Sink_result_cleaned.groupby('n').min()['computations']),
         np.log(Sto_Sink_result_cleaned.groupby('n').max()['computations']),
         color='violet', alpha = 0.2)


plt.xlabel(r'$ln(n)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=13)
plt.title("ln(number of operations) versus ln(n)",fontsize=20)

#plt.show()

plt.savefig("results/mnist_compare_stoc.pdf", format="pdf",bbox_inches = 'tight')

#4. Evaluate PDASMD-B on MNIST data


In [ ]:
# sample and rescale the images as marginal
alpha_list = []
beta_list = []
C_list = []
m_list = [7,8,10]
size_list = list(map(lambda x: x**2,m_list))
reps_each_m = 5
num_m = len(m_list)

np.random.seed(1001)
idx_list_1 = np.random.choice(range(30000),reps_each_m)
idx_list_2 = np.random.choice(range(30000,60000),reps_each_m)

#### marginal distribution list, where we reshape the mnist image to get different size marginals
for m in m_list:
    for idx1,idx2 in zip(idx_list_1,idx_list_2):
      fig1 = Image.fromarray(x_train[idx1]).resize((m,m)) ### down scale the images
      fig2 = Image.fromarray(x_train[idx2]).resize((m,m)) 
      fig1 = np.array(fig1, np.float64)
      fig2 = np.array(fig2, np.float64) 
      idx = np.where(fig1 <= .15* np.max(fig1)) 
      fig1[idx] = .15* np.max(fig1) ### add background intensity
      idx = np.where(fig2 <= .15* np.max(fig2)) 
      fig2[idx] = .15* np.max(fig2) ### add background intensity
      fig1 /= fig1.sum()
      fig2 /= fig2.sum()

      alpha_list.append(fig1.reshape(-1,1)) 
      beta_list.append(fig2.reshape(-1,1)) 
    
    C_list.append(dist_img_m(m))

#%%% parameter list
epsilons = [.1] * reps_each_m * num_m
sizes = np.repeat(size_list, reps_each_m)
idxes_c = np.repeat(range(num_m), reps_each_m)
idxes_dist = range(num_m * reps_each_m)

batch_sizes = [i for i in range(2,49,8)]

settings = []
for epsilon, size, idx_c, idx_dist in list(zip(epsilons,sizes,idxes_c,idxes_dist)):
    for batch_size in batch_sizes:
        settings.append([epsilon, size, idx_c, idx_dist, batch_size])


In [ ]:
#%% calculate epsilon solution, the goal is to compare the total computation
PDASMD_result_batch = []
i = 1
for epsilon, size, idx_c, idx_dist, batch_size in settings:
    print("Running {}/{} experiment".format(i,len(settings)))
    i += 1
    solution, computations, time = PDASMD_w_rounding(
        alpha = alpha_list[idx_dist], beta = beta_list[idx_dist],
        C = C_list[idx_c], epsilon = epsilon, Size = size, 
        inner_iters = size//batch_size, batchsize = batch_size, MD = True)
    PDASMD_result_batch.append([solution, computations, time])


#%%plot the results
PDASMD_result_batch_cleaned = pd.DataFrame([[i[0][1],i[0][2],i[1][1],i[1][4]] for i in zip(PDASMD_result_batch,settings)])
PDASMD_result_batch_cleaned.columns = ['computations','time','n','batchsize']

In [ ]:
#%% Plot the results
plt.figure(dpi=1000)

PDASMD_result_batch_mean = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).mean()['computations'])
PDASMD_result_batch_min = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).min()['computations'])
PDASMD_result_batch_max = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).max()['computations'])
#%%
for n in size_list:
    lr  = linregress(np.log(batch_sizes),PDASMD_result_batch_mean.loc[n] )
    
    plt.plot(np.log(batch_sizes), PDASMD_result_batch_mean.loc[n],label="n = {}, slope = {:.2f}".format(n,lr[0]), marker ='o',markersize = 6,lw=1)
    plt.fill_between(np.log(batch_sizes),PDASMD_result_batch_min.loc[n] ,
             PDASMD_result_batch_max.loc[n], alpha = 0.2)

plt.xlabel(r'$ln(batchsize)$',fontsize=20)
plt.ylabel(r'$ln( \# operation )$',fontsize=20) 
plt.legend(fontsize=14)
plt.title("ln(number of operations) versus ln(batchsize)",fontsize=20)

plt.savefig("results/mnist_batch_computation.pdf", format="pdf", bbox_inches = 'tight')

#%%

plt.figure(dpi=1000)
PDASMD_result_batch_mean = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).mean()['time'])
PDASMD_result_batch_min = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).min()['time'])
PDASMD_result_batch_max = np.log(PDASMD_result_batch_cleaned.groupby(['n','batchsize']).max()['time'])

for n in size_list:
    plt.plot(np.log(batch_sizes), PDASMD_result_batch_mean.loc[n],label="n = {}, slope = {:.2f}".format(n,lr[0]), marker ='o',markersize = 6,lw=1)
    plt.fill_between(np.log(batch_sizes),PDASMD_result_batch_min.loc[n] ,
             PDASMD_result_batch_max.loc[n], alpha = 0.2)
    
plt.xlabel(r'$ln(batchsize)$',fontsize=20)
plt.ylabel(r'$ln(running time)$',fontsize=20) 
plt.legend(fontsize=20)
plt.title("ln(running time) versus ln(batchsize)",fontsize=20)


plt.savefig("results/mnist_batch_time.pdf", format="pdf", bbox_inches = 'tight')

#%% save the data and results

results = {'alpha_list': alpha_list, 'beta_list': beta_list, 'C_list': C_list, 
    'PDASMD_result_batch': PDASMD_result_batch}
  
try:
    file = open('results/comparison_mnist_batch', 'wb')
    pickle.dump(results,file)
    file.close()
  
except:
    print("Something went wrong")

